In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!pip install matminer
!pip install tensorflow


In [ ]:
import sklearn
import tensorflow as tf
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf
import pandas as pd
import numpy as np
np.random.seed(44)
import os
import sys
import random
from pymatgen.core import Composition
import tensorflow as tf
from tensorflow import keras
from keras import initializers, regularizers
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [ ]:
file_path = "dataset.xlsx"
data = "dataset.xlsx"
df = pd.read_excel(file_path)
property_interest = 'thermal_conductivity(W/mK)'
print(df.head())

num_rows = df.shape[0]
print(f"The number of rows in the DataFrame is: {num_rows}")

In [ ]:
import pandas as pd
file_path = "dataset.xlsx"
try:
    data = pd.read_excel(file_path)
    print("Data loaded successfully.")
except Exception as e:
    print(f"Error loading data: {e}")
    data = None
if isinstance(data, pd.DataFrame):
    print(data.columns)
    print(data.head())
else:
    print("Data is not a valid DataFrame.")


In [ ]:
def get_compostion(c):
    try:
        return Composition(c)
    except:
        return None

In [ ]:
from pymatgen.core.composition import Composition

def get_compostion(c):
    try:
        composition = Composition(c)
        return composition
    except Exception as e:
        print(f"Error processing formula '{c}': {e}")
        return None


In [ ]:
data['composition'] = data['Formula'].apply(get_compostion)
data['temperature(K)'] = pd.to_numeric(data['temperature(K)'], errors='coerce')
data[property_interest] = pd.to_numeric(data[property_interest], errors='coerce')


In [ ]:
data.reset_index(drop=True, inplace=True)

In [ ]:
f =  MultipleFeaturizer([cf.Stoichiometry(), cf.ElementProperty.from_preset("magpie"), cf.ValenceOrbital(props=['avg']), cf.ElementFraction()]) # Featurizers

X = np.array(f.featurize_many(data['composition'], ignore_errors=True))
measuring_temp_array = np.array(data['temperature(K)']).reshape(-1,1)
X = np.hstack((X,measuring_temp_array))

y = data[property_interest].values
x_df = pd.DataFrame(X)
x_df = x_df.loc[:, x_df.std() != 0]
print(x_df.shape)

In [ ]:
# Calculate the correlation matrix
corr_matrix = x_df_no_constant.corr(method="pearson").abs()

# Preview the first 5 rows/columns of the correlation matrix
print(corr_matrix.iloc[:5, :5])


In [ ]:
# Filter the features with correlation coefficients above 0.95
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
x_df_low_corr = x_df_no_constant.drop(columns=to_drop)

# Recalculate the correlation matrix for the updated features
corr_matrix_low_corr = x_df_low_corr.corr(method="pearson").abs()




In [ ]:
X = x_df_low_corr
measuring_temp_array = np.array(data['temperature(K)']).reshape(-1,1)

In [ ]:

y = data[property_interest].values
x_df = pd.DataFrame(X)
x_df = x_df.loc[:, x_df.std() != 0]
print(x_df.shape)

(5271, 170)


In [ ]:
# Calculate standard deviation for each feature in the training data
std_dev = np.std(x_df, axis=0)

# Identify features with zero standard deviation
zero_std_dev_features = np.where(std_dev == 0)[0]
# Exclude features with zero standard deviation from both training and test data
filtered_x_df = np.delete(x_df, zero_std_dev_features, axis=1)
print(filtered_x_df.shape)

(5271, 170)


In [ ]:

from sklearn.utils import shuffle

all_values = [list(x_df.iloc[x]) for x in range(len(x_df.index))]
all_values = np.array(all_values, dtype=float)

all_labels = y.copy()

train_percent = 0.90
index_split_at = int(train_percent * len(all_labels))

all_values, all_labels = shuffle(all_values, all_labels, random_state=1)

train_values, test_values = np.split(all_values, [index_split_at])
train_labels, test_labels = np.split(all_labels, [index_split_at])

feature_mean = np.mean(train_values, axis=0)
feature_std = np.std(train_values, axis=0)

# Check for zero standard deviation before scaling
zero_std_dev_features = np.where(feature_std == 0)[0]
non_zero_std_dev_features = np.where(feature_std != 0)[0]

# Exclude features with zero standard deviation from scaling
train_values[:, non_zero_std_dev_features] = (train_values[:, non_zero_std_dev_features] - feature_mean[non_zero_std_dev_features]) / feature_std[non_zero_std_dev_features]
test_values[:, non_zero_std_dev_features] = (test_values[:, non_zero_std_dev_features] - feature_mean[non_zero_std_dev_features]) / feature_std[non_zero_std_dev_features]

print("Shape of Train Values:", train_values.shape)
print("Shape of Train Labels:", train_labels.shape)

print("Shape of Test Values:", test_values.shape)
print("Shape of Test Labels:", test_labels.shape)


In [ ]:
#neural network model
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

kernel_init = initializers.RandomNormal(seed=30)
bias_init = initializers.Zeros()

model = Sequential()
model.add(Dense(120, activation='relu', use_bias=True, input_shape=(train_values.shape[1],), kernel_initializer=kernel_init, bias_initializer=bias_init))
#model.add(Dense(120, activation='relu', use_bias=True, kernel_initializer=kernel_init, bias_initializer=bias_init))
model.add(Dense(90, activation='relu', use_bias=True, kernel_initializer=kernel_init, bias_initializer=bias_init))
model.add(Dense(60, activation='relu', use_bias=True, kernel_initializer=kernel_init, bias_initializer=bias_init))
model.add(Dense(1, activation='linear', use_bias=True, kernel_initializer=kernel_init, bias_initializer=bias_init))

optimizer = Adam()

model.compile(loss='mae', optimizer=optimizer, metrics=['mae'])

# EARLY STOPPING CRITERIAS
valmae_es = tf.keras.callbacks.EarlyStopping(monitor='val_mae', min_delta=1e-10, patience=1000, verbose=1, mode='auto', restore_best_weights=True)

# EPOCH REAL TIME COUNTER CLASS
class PrintEpNum(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        sys.stdout.flush()
        sys.stdout.write("Current Epoch: " + str(epoch+1) + " Training Loss: " + "%4f" % logs.get('loss') + '                                       \r')

EPOCHS = 30000  # Number of EPOCHS

# HISTORY Object which contains how the model learned
history = model.fit(train_values, train_labels, batch_size=90, validation_split=0.1, shuffle=False, epochs=EPOCHS, verbose=False, callbacks=[PrintEpNum(), valmae_es])

# PLOTTING HISTORY USING MATPLOTLIB
plt.figure()
plt.xlabel('Epoch')
plt.ylabel('Mean Abs Error')
plt.plot(history.epoch, np.array(history.history['mae']), label='MAE Training')
plt.plot(history.epoch, np.array(history.history['val_mae']), label='MAE Validation')
print("Loss at best epoch", min(list(np.array(history.history['mae']))))
plt.legend()
plt.savefig('training.pdf')
plt.show()

[loss, mae] = model.evaluate(train_values, train_labels, verbose=0)
print("Training Set Mean Absolute Error: {:2.4f} units".format(mae))

[loss, mae] = model.evaluate(test_values, test_labels, verbose=0)
print("Testing Set Mean Absolute Error: {:2.4f} units".format(mae))

In [ ]:
#randomforeset model
from sklearn.metrics import mean_absolute_error, r2_score

# Predictions on the test set
test_predictions = model.predict(test_values)

# Calculate Mean Absolute Error (MAE) on the test set
mae_test = mean_absolute_error(test_labels, test_predictions)
print("Test Set Mean Absolute Error: {:2.4f} units".format(mae_test))

# Calculate R-squared (R^2) on the test set
r2_test = r2_score(test_labels, test_predictions)
print("Test Set R-squared (R^2): {:2.4f}".format(r2_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# Create and train the RandomForestRegressor
model = RandomForestRegressor(n_estimators=500, random_state=0)
model.fit(train_values, train_labels)

# Predict on the test set
test_predictions = model.predict(test_values)

# Calculate MAE and R-squared for the test set
mae = mean_absolute_error(test_labels, test_predictions)
r2 = r2_score(test_labels, test_predictions)

print("Mean Absolute Error on Test Set:", mae)
print("R-squared on Test Set:", r2)

# Plotting
plt.figure(figsize=(10, 6))

# Plot actual vs. predicted values
plt.scatter(test_labels, test_predictions, color='blue', label='Predicted Values')

# Plot a 45-degree line for reference
plt.plot([min(test_labels), max(test_labels)], [min(test_labels), max(test_labels)], '--', color='red', label='45-Degree Line')

plt.title('Actual vs. Predicted Values on Test Set')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()


In [ ]:
#xgboost model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Shuffle and split the data
x_train, x_test, y_train, y_test = train_test_split(x_df, y, test_size=0.1, random_state=40)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 2, 3]
}

# Create an XGBoost regressor
xgb_model = XGBRegressor(random_state=40)

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(x_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Create a new XGBoost regressor with the best hyperparameters
best_xgb_model = XGBRegressor(**best_params, random_state=40)

# Train the model
best_xgb_model.fit(x_train, y_train)

# Make predictions on the test set
xgb_predictions = best_xgb_model.predict(x_test)

# Evaluate the model
mae = mean_absolute_error(y_test, xgb_predictions)
r2 = r2_score(y_test, xgb_predictions)

print(f"XGBoost MAE: {mae}")
print(f"XGBoost R-squared: {r2}")
